Steps:
- Scarping Job postings with Selenium and bs4
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [1]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn


Select wanted url

In [2]:
city = str(input('Please enter the city you want to work in'))
state = str(input('Please enter the state you want to work in'))
country = str(input('Please enter the country you want to work in'))
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location='+ city + '%2C%20'+ state +'%2C%20'+ country

no_of_jobs = 50

url



'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Oslo%2C%20%2C%20Norway'

open windows with url stated above in

In [3]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [4]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [5]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [6]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Data Analyst
Attensi


Create job container

In [7]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

41


create for loop to look for all jobs

In [8]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 41 jobs.


In [9]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [10]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

{'Title': ['Data Analyst', 'Data Analyst - Aftenposten', 'Data Analyst', 'Business Analyst', 'Data Analyst - Flexible Office Homeoffice Remote - Talent', 'Data Scientist (Product Analyst)', 'Business analyst - New revenue streams', 'Data Scientist - Refinery operations', 'Test Analyst', 'Support Analyst', 'Data Engineer', 'Business Analyst', 'Statistician', 'Dataanalytiker', 'Business Analyst/Adoption Responsible, Digital Collaboration', 'Talent & Organization - Analysts, Consultants, Managers - Norway ', 'Senior Analyst M&A', 'Data Analyst - Oslo', 'Senior Customer Insight Analyst', 'DigitalBCG | Associates to Principals ', 'Analytical Programmer / Data Scientist Role in Oslo', 'Analytics Foundation - BI Developer', 'Power BI Developer', 'TDM Analyst', 'Har du interesse for programmering og analyse? Bli Dataanalytiker på 12 uker!', 'Healthcare Data Analyst - Talent', 'Har du interesse for programmering og analyse? Bli Dataanalytiker på 12 uker! - Oslo', 'Junior Analytical Program Mana

In [11]:
df = pd.DataFrame(ml)
df.head(1)

,Title,Company,Locations,Post Date
0,Data Analyst,Attensi,"Oslo, Oslo, Norway",2021-01-04


Separate City from State and Country


In [12]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [13]:
new2 = new.rename(columns = {0: 'City2'})

In [14]:
new2


,City2,1,2
0,Oslo,Oslo,Norway
1,Oslo,Oslo,Norway
2,Oslo,Oslo,Norway
3,Oslo,Norway,None
4,Oslo,Oslo,Norway
5,Oslo,Oslo,Norway
6,Oslo,Norway,None
7,Oslo,Norway,None
8,Oslo,Norway,None
9,Oslo,Oslo,Norway


merge df and new, drop Locations as it is redundant now

In [15]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

,Title,Company,Post Date,City2
0,Data Analyst,Attensi,2021-01-04,Oslo
1,Data Analyst - Aftenposten,Schibsted,2021-01-12,Oslo
2,Data Analyst,Unacast,2020-12-21,Oslo
3,Business Analyst,ITR Partners,2021-01-22,Oslo


In [16]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new5 = new4.City
new5.head()

0    Oslo
1    Oslo
2    Oslo
3    Oslo
4    Oslo
Name: City, dtype: object

In [17]:
df4 = df2.join(new5, how = 'left')
df4.drop(columns = ["City2"], inplace = True)
df4.set_index('Title', inplace = True)
df4

,Company,Post Date,City
Title,,,
Data Analyst,Attensi,2021-01-04,Oslo
Data Analyst - Aftenposten,Schibsted,2021-01-12,Oslo
Data Analyst,Unacast,2020-12-21,Oslo
Business Analyst,ITR Partners,2021-01-22,Oslo
Data Analyst - Flexible Office Homeoffice Remote - Talent,Talent,2021-01-23,Oslo
Data Scientist (Product Analyst),UserTesting,2020-10-05,Oslo
Business analyst - New revenue streams,NoA Consulting,2021-01-13,Oslo
Data Scientist - Refinery operations,Rystad Energy,2021-01-05,Oslo
Test Analyst,Infosys,2021-01-06,Oslo


create CSV, with todays Date

In [18]:
date_object = datetime.date.today()
date = str(date_object) + '_'
date

df4.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/' + date + city + '_' + country + '.csv')

concat new jobs to existing list

In [19]:
final_df = pd.read_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')
final_df.set_index('Title', inplace = True)
final_df.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


In [20]:
dfWIP = pd.concat([final_df, df4], axis=0)
dfWIP.head()

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt


drop duplicated rows

In [21]:
new_final_df = dfWIP.drop_duplicates()
new_final_df

,Company,Post Date,City
Title,,,
Data Analyst,EPAM Systems,2021-01-15,Zurich
Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
Business Analyst,Tata Consultancy Services,2021-01-14,Zurich
Business Intelligence Analyst (80%),Spitex Zürich Limmat,2021-01-22,Zurich
"Business Analyst (m/w), Zurich",MoneyPark,2021-01-22,Schlatt
...,...,...,...
Data Scientist (Product Analyst) - Oslo,UserTesting,2020-12-08,Oslo
Product Analyst - Oslo,VEV AS,2021-01-23,Oslo
Product Analyst - Oslo,Vev,2021-01-24,Oslo


In [22]:
new_final_df.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')

